# Day 5 - Lab 1: Tool-Using Agents (Solution)

**Objective:** Build agents that can use external tools to accomplish tasks they cannot perform on their own, using both LangChain and custom Python functions.

**Introduction:**
This solution notebook provides the complete code and explanations for building your first agents. It dives into the LangChain framework for control and flexibility, culminating in a multi-tool agent.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Explanation:**
We ensure all necessary libraries for this lab are installed. `langchain` and its related packages provide the core framework for building agents, while `tavily-python` is the SDK for the Tavily Search API, which our agent will use as a tool.

In [7]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# This helper will install packages if they are not found
import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('langchain')
install_if_missing('langchain_community')
install_if_missing('langchain_openai')
install_if_missing('tavily')

from utils import setup_llm_client
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Building a LangChain Agent with One Tool

**Explanation:**
LangChain provides a modular and flexible way to build agents. 
1.  **Tool:** We instantiate `TavilySearchResults`, which is a pre-built LangChain component that knows how to call the Tavily API.
2.  **Prompt:** The prompt is crucial. `ChatPromptTemplate.from_messages` creates a template for the conversation. The key part is the `("placeholder", "{agent_scratchpad}")`. The `agent_scratchpad` is a special variable where the agent's internal monologue (its thoughts, tool calls, and tool outputs) is stored. This allows the agent to reason about its actions.
3.  **Agent:** `create_tool_calling_agent` binds the LLM, the tools, and the prompt together into a runnable agent.
4.  **AgentExecutor:** This is the runtime for the agent. It takes the agent and the tools and handles the logic of calling the agent, parsing its output to see if it wants to use a tool, executing the tool, and passing the result back to the agent to continue its reasoning.

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# We need to use a LangChain LLM wrapper
llm = ChatOpenAI(model=model_name)

# 1. Instantiate the Tavily search tool
search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

# 2. Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 3. Create the agent
agent = create_tool_calling_agent(llm, tools, prompt)

# 4. Create the AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 5. Invoke the agent with a question
question = "What was the score of the last Super Bowl?"
result = agent_executor.invoke({"input": question})

print(f"\nFinal Answer: {result['output']}")



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'last Super Bowl score'}`


[{'title': 'List of Super Bowl champions - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/List_of_Super_Bowl_champions', 'content': 'Super Bowl championships | Game | Date (Season) | Winning team | Score | Losing team | Venue | City | Attendance | Referee | Ref. |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| I ( | January 15, 1967 (1966 AFL/1966 NFL) | Green Bay Packersn (1, 1–0) | 35–10 | Kansas City Chiefsa (1, 0–1) | Los Angeles Memorial Coliseum | Los Angeles, California( | 61,946 | Norm Schachter | ( | [...] | XLIV | February 7, 2010 (2009) | New Orleans SaintsN (1, 1–0) | 31–17 | Indianapolis ColtsA (4, 2–2) | Sun Life Stadium (5)( | Miami Gardens, Florida (10)( | 74,059 | Scott Green "Scott Green (American football official)") | ( |\n| XLV | February 6, 2011 (2010) | Green Bay PackersN (5, 4–1)( | 31–25 | Pittsburgh SteelersA (8, 6–2

### Challenge 2 (Intermediate): Building a Multi-Tool Agent

**Explanation:**
This challenge demonstrates the core reasoning power of an agent. 
1.  **`@tool` Decorator:** LangChain provides a simple `@tool` decorator to turn any Python function into a tool that an agent can use. The function's docstring is very important, as the agent uses it to understand what the tool does.
2.  **Tool List:** We create a new list containing both our custom `multiply` tool and the pre-built `TavilySearchResults` tool.
3.  **Reasoning:** When the agent receives a query, it looks at the user's question and the docstrings of all available tools. It then makes a decision about which tool, if any, is the most appropriate for the task. This ability to choose the right tool for the job is the fundamental capability of an agent. For "25 * 48", it will see the "Multiplies two integers" docstring and choose the `multiply` tool. For "Who is the CEO of Apple?", it will recognize that it needs external information and choose the search tool.

In [4]:
from langchain_core.tools import tool

# 1. Define your custom calculator tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers together."""
    return a * b

# 2. Create the new list of tools
multi_tool_list = [TavilySearchResults(max_results=2), multiply]

# 3. Create the new multi-tool agent and executor
multi_tool_agent = create_tool_calling_agent(llm, multi_tool_list, prompt)
multi_tool_executor = AgentExecutor(agent=multi_tool_agent, tools=multi_tool_list, verbose=True)

# 4. Invoke the agent with a math question
math_question = "What is 25 * 48?"
math_result = multi_tool_executor.invoke({"input": math_question})
print(f"\nQuery: {math_question}\nFinal Answer: {math_result['output']}\n")

# 5. Invoke the agent with a search question
search_question = "Who is the current CEO of Apple?"
search_result = multi_tool_executor.invoke({"input": search_question})
print(f"\nQuery: {search_question}\nFinal Answer: {search_result['output']}")



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 25, 'b': 48}`


1200The product of \(25 \times 48\) is 1200.

> Finished chain.

Query: What is 25 * 48?
Final Answer: The product of \(25 \times 48\) is 1200.



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current CEO of Apple 2023'}`


[{'title': 'Tim Cook - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Tim_Cook', 'content': 'Wikipedia\nThe Free Encyclopedia\n\n## Contents\n\n# Tim Cook\n\nPage semi-protected\n\n| Tim Cook | |\n| --- | --- |\n| Cook in 2023 | |\n| Born | Timothy Donald Cook   (1960-11-01) November 1, 1960 (age 64)  Mobile, Alabama, U.S. |\n| Education | Auburn University (BS) Duke University (MBA) |\n| Occupation | Business executive |\n| Employer | Apple Inc. (1998–present) |\n| Title | CEO of Apple Inc. (2011–present) |\n| Board member of | Nike, Inc. |\n| Website | apple.com |\n| Signature | |\n|  | | [...] Timothy Donald Cook (born Nove

### Challenge 3 (Advanced): Improving Tool Selection with Better Docstrings

**Explanation:**
This challenge highlights the importance of 'prompt engineering' your tool's docstrings. The agent's reasoning process is heavily influenced by the descriptions of the tools it has available. A vague docstring might lead the agent to ignore a tool or use it incorrectly. A highly descriptive docstring, as shown in the solution, gives the agent a much clearer signal about the tool's purpose, leading to more reliable and efficient tool selection.

In [6]:
# 1. Redefine your custom tool with a more descriptive docstring.
@tool
def better_multiply(a: int, b: int) -> int:
    """Use this tool for mathematical calculations involving multiplication. It takes two integers as input and returns their product."""
    return a * b

# 2. Re-create the tool list, agent, and executor with the improved tool.
improved_tool_list = [TavilySearchResults(max_results=2), better_multiply]
improved_agent = create_tool_calling_agent(llm, improved_tool_list, prompt)
improved_executor = AgentExecutor(agent=improved_agent, tools=improved_tool_list, verbose=True)

# 3. Invoke the agent again.
math_question = "What is 25 * 48?"
improved_math_result = improved_executor.invoke({"input": math_question})
print(f"\nQuery: {math_question}\nFinal Answer: {improved_math_result['output']}")



> Entering new AgentExecutor chain...

Invoking: `better_multiply` with `{'a': 25, 'b': 48}`


1200The product of 25 and 48 is 1200.

> Finished chain.

Query: What is 25 * 48?
Final Answer: The product of 25 and 48 is 1200.


## Lab Conclusion

Congratulations! You have successfully built your first AI agents. You've learned how to give agents tools to extend their capabilities and, most importantly, how to build an agent that can reason about which tool to use for a specific task. This is the foundational skill for all advanced agentic workflows we will explore in the coming days.

> **Key Takeaway:** An agent's ability to reason and act depends on its understanding of its tools. The most critical part of creating a custom tool is writing a clear, descriptive docstring that tells the agent exactly what the tool does and when to use it.